# 규제 선형 모델

- 다항식이 복잡해지면서 회귀 계수값이 크게 설정되고 과대 적합을 유발함
    -> 훈련 데이터에서는 높은 성능을 보이지만, 테스트 데이터에서는 낮은 성능을 보임

- 선형 모델의 목적은 비용 함수를 최소화 하는 것
- 이때, 규제 선형 모델은 비용 함수를 최소화 하는 데에 패널티(alpha)를 적용해 회귀 계수의 크기를 제한함

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Rige 회귀
- L2 규제를 적용한 회귀 모델
- 회귀 계수들의 제곱합에 alhpah 패널치를 곱해 비용 산정에 반영
- alpha 값이 커지면 회귀 계수가 작아짐 -> 과대적합 방지

In [13]:
from sklearn.model_selection import train_test_split

boston_df=pd.read_csv('./data/boston_housing_train.csv')

X=boston_df.drop('MEDV',axis=1)
y=boston_df['MEDV']

X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [14]:
from sklearn.linear_model import Ridge
reg=Ridge()

reg.fit(X_train,y_train)

print(reg.score(X_train,y_train),reg.score(X_test,y_test))

print(reg.coef_)

0.7461161787884155 0.678974832784608
[-0.12383039  0.03139178  0.01767668  2.54277179 -8.77249222  4.37980204
 -0.01537349 -1.29086084  0.24406848 -0.01082435 -0.83346553  0.01348642
 -0.53435396]


In [15]:
# reg.alpha

In [16]:
# 다른 평가 지표
from sklearn.metrics import mean_squared_error,root_mean_squared_error,mean_absolute_error,r2_score

# 평가지표들
def evauate_regression(y_true,y_pred):
    mse=mean_squared_error(y_true,y_pred)
    rmse=root_mean_squared_error(y_true,y_pred)
    mae=mean_absolute_error(y_true,y_pred)
    r2=r2_score(y_true,y_pred)
    print(f"'MSE': {mse}, 'RMSE' : {rmse}, 'MAE' : {mae}, 'R2' : {r2}")

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

model=Pipeline([
    ('poly',PolynomialFeatures(include_bias=False,degree=2)),
    ('reg',Ridge(alpha=0.01))
])

model.fit(X_train,y_train)
print(model.score(X_train,y_train),model.score(X_test,y_test))
evauate_regression(y_test,model.predict(X_test))

0.9438738432321337 0.7872498200566541
'MSE': 14.898287413368152, 'RMSE' : 3.8598299720801372, 'MAE' : 2.536591106635486, 'R2' : 0.7872498200566541
